# NLP

In [ ]:
import os
from nltk.tag import UnigramTagger
from nltk.corpus import treebank
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
import pandas as pd

In [ ]:
train_sents = treebank.tagged_sents()[:3000]
tagger = UnigramTagger(train_sents)

In [ ]:
path_neg=r'D:\OneDrive - De Vinci\ESILV\A5\Machine_learning_NLP\wp2\txt_sentoken\neg'
path_pos=r'D:\OneDrive - De Vinci\ESILV\A5\Machine_learning_NLP\wp2\txt_sentoken\pos'

path=r'twitter_data_161122.csv'

pos_reviews =os.listdir(path_pos)
neg_reviews =os.listdir(path_neg)

In [ ]:
df = pd.read_csv(path,sep=',',encoding='utf-8')
df.head()

In [ ]:
test=df.loc[0]['Text']
print(test)

In [ ]:
def Predire(text):
    '''
    if type=='adverbe':
        Ltag=['RB','RBR','RBS','RBW']
        senti='r'
    elif type=='verbe':
        Ltag=['VB','VBD','VBG','VBN','VBP','VBZ']
        senti='v'
    '''


    texte=word_tokenize(text)
    L=set()
    tag=tagger.tag(texte)

    for t in tag:
        L.add(t[0])
        #print(t[0])

    pos_score=0
    neg_score=0

    for adv in L:
        adv_senti = list(swn.senti_synsets(adv))
        if len(adv_senti)!=0:
            #print(adv_senti[0])
            pos_score+=adv_senti[0].pos_score()
            neg_score+=adv_senti[0].neg_score()


    res=''
    if pos_score > neg_score:
        res='pos'
    elif pos_score < neg_score:
        res='neg'

    return res,pos_score,neg_score

In [ ]:
Predire(test,'adverbe')

In [ ]:
for i in range(10):
    print('--------------')
    print(df.loc[i]['Text'])
    print(Predire(df.loc[i]['Text']))
    print('--------------')

In [ ]:
pos=0
neg=0
neutre=0
for index,row in df.iterrows():
    if Predire(row['Text'])[0] == 'pos':
        pos+=1
    elif Predire(row['Text'])[0] == 'neg':
        neg+=1
    else:
        neutre+=1

print('total= ', len(df))
print(pos,neg,neutre)

# XLNET

In [ ]:
#!pip install transformers
!pip install -q -U watermark
!pip install SentencePiece

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

In [ ]:
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report#, accuracy
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

from torch import nn, optim
#from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
df = pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Tests_modeles\Train.csv')
df.head()

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)
df.head(20)

In [ ]:
import re
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
df['text'] = df['text'].apply(clean_text)

In [ ]:
rcParams['figure.figsize'] = 8, 6
sns.countplot(df['label'])
plt.xlabel('review score');

In [ ]:
df['label'].value_counts()

In [ ]:
class_names = ['negative', 'positive']

In [ ]:
from transformers import XLNetTokenizer, XLNetModel
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
input_txt = "India is my country. All Indians are my brothers and sisters"
encodings = tokenizer.encode_plus(input_txt, add_special_tokens=True, max_length=16, return_tensors='pt', return_token_type_ids=False, return_attention_mask=True, pad_to_max_length=False)